In [1]:
import csv
import json
import sys
import pandas as pd
from pathlib import Path
import requests
import pprint
sys.path.insert(0, 'C:/Users/sam/Desktop/ScrapeTest') 
from cookie_monster import COOKIE
import demjson
pd.set_option("display.max_rows", None, "display.max_columns", None)

user_cookie = dict()

for item in COOKIE.split("; "):
    k, v = item.split("=")
    user_cookie[k] = v

urllist = ["https://lpl.qq.com/es/stats.shtml?bmid=7325",
        "https://lpl.qq.com/es/stats.shtml?bmid=6909"]

test_url = "https://lpl.qq.com/es/stats.shtml?bmid=6909"


full_headers_dict = {"summonerName":"player", "championId": "champion", "magicDamageDealt":"totalmagicdamagedealt", "physicalDamageDealt":"totalphysicaldamagedealt", "physicalDamageTaken":"physicaldamagetaken", "kills":"kills","deaths":"deaths","assists":"assists", 'firstBloodKill': 'firstblood', 'totalDamageDealtToChampions': 'totaldamagetochampions', 'physicalDamageDealtToChampions': 'physicaldamagetochampions', 'magicDamageDealtToChampions': 'magicdamagetochampions', 'trueDamageDealtToChampions': 'truedamagetochampions', 'damageDealtToObjectives': 'totaldamagetoobjectives', 'damageDealtToTurrets': 'totaldamagetoturrets', 'totalHeal': 'damagehealed', 'totalDamageTaken': 'damagetaken', 'magicalDamageTaken': 'magicdamagetaken', 'wardsKilled': 'wardsdestroyed', 'sightWardsBoughtInGame': 'stealthwardspurchased', 'visionWardsBoughtInGame': 'controlwardspurchased', 'totalMinionsKilled': 'minionskilled', 'neutralMinionsKilledTeamJungle': "neutralminionskilledinteam'sjungle", 'neutralMinionsKilledEnemyJungle': 'neutralminionskilledinenemyjungle', 'killingSprees': 'killingsprees', 'longestTimeSpentLiving': 'longesttimespentliving', 'damageSelfMitigated': 'selfmitigateddamage', 'visionScore': 'visionscore', 'timeCCingOthers': 'timeccingothers', 'turretKills': 'totalTurretKills', 'inhibitorKills': 'totalInhibitorKills', 'totalTimeCrowdControlDealt': 'totaltimeapplyingcc', 'firstBloodAssist': 'firstbloodassist', 'firstTowerKill': 'firsttowerkill', 'firstTowerAssist': 'firsttowerassist', 'firstInhibitorKill': 'firstinhibkill', 'firstInhibitorAssist': 'firstinhibassist', 'doubleKills': 'doublekills', 'tripleKills': 'triplekills', 'quadraKills': 'quadrakills', 'pentaKills': 'pentakills', 'goldEarned':'goldearned', 'goldSpent': 'goldspent', "largestCriticalStrike":"largestcriticalstrike", "largestKillingSpree":"largestkillingspree", "largestMultiKill":"largestmultikill", "teamId":'side', 'gameId':'gameId'}
lpl_full_headers_dict = {"name": "player","hero":"champion","kill":"kills","death":"deaths","assist":"assists",'firstBlood': 'firstblood', 'totalDamageToChamp': 'totaldamagetochampions', 'pDamageToChamp': 'physicaldamagetochampions', 'mDamageDealtToChamp': 'magicdamagetochampions', 'trueDamageDealtToChampions': 'truedamagetochampions', 'damageDealtToObjectives': 'totaldamagetoobjectives', 'damageDealtToTurrets': 'totaldamagetoturrets', 'totalHeal': 'damagehealed', 'totalDamageTaken': 'damagetaken', 'mDamageTaken': 'magicdamagetaken', 'wardsKilled': 'wardsdestroyed', 'sightWardsBoughtInGame': 'stealthwardspurchased', 'visionWardsBought': 'controlwardspurchased', 'lasthit': 'minionskilled', 'neutralKilledTJungle': "neutralminionskilledinteam'sjungle", 'neutralKilledEJungle': 'neutralminionskilledinenemyjungle', 'killingSprees': 'killingsprees', 'longestTimeSpentLiving': 'longesttimespentliving', 'damageSelfMitigated': 'selfmitigateddamage', 'visionScore': 'visionscore', 'timeCCingOthers': 'timeccingothers', 'towerKills': 'totalTurretKills', 'inhibitorKills': 'totalInhibitorKills', 'totalTimeCrowdControlDealt': 'totaltimeapplyingcc', 'firstBloodAssist': 'firstbloodassist', 'firstTowerKill': 'firsttowerkill', 'firstTowerAssist': 'firsttowerassist', 'firstInhibitorKill': 'firstinhibkill', 'firstInhibitorAssist': 'firstinhibassist', 'dKills': 'doublekills', 'tKills': 'triplekills', 'qKills': 'quadrakills', 'pKills': 'pentakills', 'side':'side', 'gold':'goldearned', 'GoldSpent': 'goldspent', "pDamageDealt":"totalphysicaldamagedealt", "pDamageTaken":"physicaldamagetaken", 'mDamageDealt':'totalmagicdamagedealt', 'largestCriticalStrike':'largestcriticalstrike',  "largestKillingSpree":"largestkillingspree", "largestMultiKill":"largestmultikill", 'game-id':'gameId'}
scrape_headers = full_headers_dict.keys()
lpl_scrape_headers = lpl_full_headers_dict.keys()
table_headers = full_headers_dict.values()


#This is pulling the champion data.
champ_dd = requests.get("https://ddragon.leagueoflegends.com/cdn/11.8.1/data/en_US/champion.json", cookies=user_cookie)
champ_json = champ_dd.json()
champ_name = champ_json["data"] #this creates a list of all the champions
champ_key_dict = {}
for i in champ_name:
    champ_key_dict.update({champ_json["data"][i]["key"]:i})
#pprint.pprint(champ_dd.json())
#control = {'side':'left', 'count':0, 'records':0}
#json_data['bMatchId']
#jd = json.loads(json_data["sMatchInfo"][0]['battleInfo']['BattleData'])
#jd['left']['players']

def lpl_url_for_request_scraping(raw_url, test):
    base_url = "https://lpl.qq.com/web201612/data/LOL_MATCH_DETAIL_"
    js = ".js"
    match_num = raw_url[-4:]
    new_url = base_url + match_num + js
    if test == True:
        pprint.pprint(new_url)
    return (new_url)

#Get the lpl json data. 
def lpl_get_match_data(url, test):
    good_url = lpl_url_for_request_scraping(url, False)
    json_file = requests.get(good_url)
    json_content = json.loads(json_file.text[12:-1]) # or demjson.decode(json_file.content[12:-1])
    #assert json_file.status_code == 200
    if test == True:
        pprint.pprint(json_content)
    return(json_content)

def lpl_build_dataframe(input_match_data, test):
    jd = json.loads(input_match_data["sMatchInfo"][0]['battleInfo']['BattleData'])
    #'bMatchId' is what matches the URL
    side = 'left'
    count = 0
    records = 0
    while records < 10:
        if count == 5:
            count = 0
            side = 'right'
        else:
            player_info = jd[side]['players'][count]
            champion_name = champ_key_dict[str(player_info['hero'])]
            gameId = json.loads(input_match_data['bMatchId'])
            player_info.update({"hero": champion_name , "gameId": gameId})
            if count == 0 and records == 0:
                df = pd.DataFrame([player_info])
                records = records + 1
                count = count + 1
            else:
                new_df = pd.DataFrame([player_info])
                count = count + 1
                records = records + 1
                df = df.append(new_df)
    if test == True:
        pprint.pprint(df) 
    return (df)

def lpl_prune_dataframe(input_raw_dataframe, test):
    bad_df = input_raw_dataframe
    all_columns = bad_df.columns.values.tolist()
    bad_keys = []
    for i in all_columns:
        if i not in lpl_scrape_headers:
            bad_keys.append(i)
    good_df = bad_df.drop(bad_keys, axis=1)
    good_df['firstBlood'] = good_df['firstBlood'].apply(lambda x: 1 if x else 0)
    if test == True:
        pprint.pprint(good_df)    
    return (good_df)


y = lpl_get_match_data(test_url, False)
z = lpl_build_dataframe(y, False)
aa = lpl_prune_dataframe(z, False)
bb = aa.rename(columns = lpl_full_headers_dict)
display(bb)

,kills,deaths,assists,goldearned,minionskilled,champion,player,totaldamagetochampions,physicaldamagetochampions,magicdamagetochampions,totalphysicaldamagedealt,totalmagicdamagedealt,largestcriticalstrike,damagehealed,damagetaken,physicaldamagetaken,magicdamagetaken,neutralminionskilledinteam'sjungle,neutralminionskilledinenemyjungle,wardsdestroyed,controlwardspurchased,doublekills,triplekills,quadrakills,pentakills,totalTurretKills,totalInhibitorKills,killingsprees,largestmultikill,largestkillingspree,firstblood
0,0,0,2,10711,276,Karma,TES369,18444,1151,17292,26467,122465,0,4249,11374,5898,5332,4,0,10,3,0,0,0,0,0,0,0,0,0,0
0,1,2,1,10106,197,Pantheon,TESKarsa,8585,7367,987,166926,10791,0,14062,29694,20572,8904,115,3,12,12,0,0,0,0,0,0,0,1,0,0
0,2,1,0,13284,343,Syndra,TESknight,13105,224,12881,16454,194143,0,1860,8972,3300,5609,24,10,13,1,0,0,0,0,1,0,1,1,2,1
0,0,2,2,13742,360,Samira,TESJackeyLove,20740,18828,822,188414,16933,468,7155,24805,12647,11845,19,0,18,5,0,0,0,0,2,0,0,0,0,0
0,0,4,1,6774,54,Leona,TESZhuo,2441,671,1267,3495,2701,0,1093,12588,4418,8043,0,0,30,19,0,0,0,0,0,0,0,0,0,0
0,1,0,6,13468,316,Aatrox,SNBin,7861,7861,0,198925,0,0,17397,34597,19408,14720,18,4,4,7,0,0,0,0,1,0,0,1,0,0
0,1,1,6,10867,205,Nidalee,SNSofM,10724,533,9974,20370,139802,0,23808,28808,18012,10384,122,8,27,13,0,0,0,0,2,0,0,1,0,0
0,2,0,5,12602,299,Viktor,SNAngel,17497,24,17472,17503,165789,0,1826,9827,3699,5860,18,0,8,5,0,0,0,0,0,1,1,1,2,0
0,5,0,3,16176,359,Kaisa,SNhuanfeng,18459,11886,5925,213184,24562,545,6272,15707,9830,5696,41,0,12,5,1,0,0,0,3,0,1,2,5,0
0,0,2,8,7407,36,Galio,SNON,3865,56,3808,954,14561,0,1651,11526,9638,1394,0,0,10,16,0,0,0,0,0,0,0,0,0,0


In [10]:
bb['test']="test value"
bb

,kills,deaths,assists,goldearned,minionskilled,champion,player,totaldamagetochampions,physicaldamagetochampions,magicdamagetochampions,totalphysicaldamagedealt,totalmagicdamagedealt,largestcriticalstrike,damagehealed,damagetaken,physicaldamagetaken,magicdamagetaken,neutralminionskilledinteam'sjungle,neutralminionskilledinenemyjungle,wardsdestroyed,controlwardspurchased,doublekills,triplekills,quadrakills,pentakills,totalTurretKills,totalInhibitorKills,killingsprees,largestmultikill,largestkillingspree,firstblood,test
0,0,0,2,10711,276,Karma,TES369,18444,1151,17292,26467,122465,0,4249,11374,5898,5332,4,0,10,3,0,0,0,0,0,0,0,0,0,0,test value
0,1,2,1,10106,197,Pantheon,TESKarsa,8585,7367,987,166926,10791,0,14062,29694,20572,8904,115,3,12,12,0,0,0,0,0,0,0,1,0,0,test value
0,2,1,0,13284,343,Syndra,TESknight,13105,224,12881,16454,194143,0,1860,8972,3300,5609,24,10,13,1,0,0,0,0,1,0,1,1,2,1,test value
0,0,2,2,13742,360,Samira,TESJackeyLove,20740,18828,822,188414,16933,468,7155,24805,12647,11845,19,0,18,5,0,0,0,0,2,0,0,0,0,0,test value
0,0,4,1,6774,54,Leona,TESZhuo,2441,671,1267,3495,2701,0,1093,12588,4418,8043,0,0,30,19,0,0,0,0,0,0,0,0,0,0,test value
0,1,0,6,13468,316,Aatrox,SNBin,7861,7861,0,198925,0,0,17397,34597,19408,14720,18,4,4,7,0,0,0,0,1,0,0,1,0,0,test value
0,1,1,6,10867,205,Nidalee,SNSofM,10724,533,9974,20370,139802,0,23808,28808,18012,10384,122,8,27,13,0,0,0,0,2,0,0,1,0,0,test value
0,2,0,5,12602,299,Viktor,SNAngel,17497,24,17472,17503,165789,0,1826,9827,3699,5860,18,0,8,5,0,0,0,0,0,1,1,1,2,0,test value
0,5,0,3,16176,359,Kaisa,SNhuanfeng,18459,11886,5925,213184,24562,545,6272,15707,9830,5696,41,0,12,5,1,0,0,0,3,0,1,2,5,0,test value
0,0,2,8,7407,36,Galio,SNON,3865,56,3808,954,14561,0,1651,11526,9638,1394,0,0,10,16,0,0,0,0,0,0,0,0,0,0,test value


In [14]:
full_headers_dict = {"summonerName":"player", "championId": "champion", "magicDamageDealt":"totalmagicdamagedealt", "physicalDamageDealt":"totalphysicaldamagedealt", "physicalDamageTaken":"physicaldamagetaken", "kills":"kills","deaths":"deaths","assists":"assists", 'firstBloodKill': 'firstblood', 'totalDamageDealtToChampions': 'totaldamagetochampions', 'physicalDamageDealtToChampions': 'physicaldamagetochampions', 'magicDamageDealtToChampions': 'magicdamagetochampions', 'trueDamageDealtToChampions': 'truedamagetochampions', 'damageDealtToObjectives': 'totaldamagetoobjectives', 'damageDealtToTurrets': 'totaldamagetoturrets', 'totalHeal': 'damagehealed', 'totalDamageTaken': 'damagetaken', 'magicalDamageTaken': 'magicdamagetaken', 'wardsKilled': 'wardsdestroyed', 'sightWardsBoughtInGame': 'stealthwardspurchased', 'visionWardsBoughtInGame': 'controlwardspurchased', 'totalMinionsKilled': 'minionskilled', 'neutralMinionsKilledTeamJungle': "neutralminionskilledinteam'sjungle", 'neutralMinionsKilledEnemyJungle': 'neutralminionskilledinenemyjungle', 'killingSprees': 'killingsprees', 'longestTimeSpentLiving': 'longesttimespentliving', 'damageSelfMitigated': 'selfmitigateddamage', 'visionScore': 'visionscore', 'timeCCingOthers': 'timeccingothers', 'turretKills': 'totalTurretKills', 'inhibitorKills': 'totalInhibitorKills', 'totalTimeCrowdControlDealt': 'totaltimeapplyingcc', 'firstBloodAssist': 'firstbloodassist', 'firstTowerKill': 'firsttowerkill', 'firstTowerAssist': 'firsttowerassist', 'firstInhibitorKill': 'firstinhibkill', 'firstInhibitorAssist': 'firstinhibassist', 'doubleKills': 'doublekills', 'tripleKills': 'triplekills', 'quadraKills': 'quadrakills', 'pentaKills': 'pentakills', 'goldEarned':'goldearned', 'goldSpent': 'goldspent', "largestCriticalStrike":"largestcriticalstrike", "largestKillingSpree":"largestkillingspree", "largestMultiKill":"largestmultikill", "teamId":'side', 'gameId':'gameId'}
lpl_full_headers_dict = {"name": "player","hero":"champion","kill":"kills","death":"deaths","assist":"assists",'firstBlood': 'firstblood', 'totalDamageToChamp': 'totaldamagetochampions', 'pDamageToChamp': 'physicaldamagetochampions', 'mDamageDealtToChamp': 'magicdamagetochampions', 'trueDamageDealtToChampions': 'truedamagetochampions', 'damageDealtToObjectives': 'totaldamagetoobjectives', 'damageDealtToTurrets': 'totaldamagetoturrets', 'totalHeal': 'damagehealed', 'totalDamageTaken': 'damagetaken', 'mDamageTaken': 'magicdamagetaken', 'wardsKilled': 'wardsdestroyed', 'sightWardsBoughtInGame': 'stealthwardspurchased', 'visionWardsBought': 'controlwardspurchased', 'lasthit': 'minionskilled', 'neutralKilledTJungle': "neutralminionskilledinteam'sjungle", 'neutralKilledEJungle': 'neutralminionskilledinenemyjungle', 'killingSprees': 'killingsprees', 'longestTimeSpentLiving': 'longesttimespentliving', 'damageSelfMitigated': 'selfmitigateddamage', 'visionScore': 'visionscore', 'timeCCingOthers': 'timeccingothers', 'towerKills': 'totalTurretKills', 'inhibitorKills': 'totalInhibitorKills', 'totalTimeCrowdControlDealt': 'totaltimeapplyingcc', 'firstBloodAssist': 'firstbloodassist', 'firstTowerKill': 'firsttowerkill', 'firstTowerAssist': 'firsttowerassist', 'firstInhibitorKill': 'firstinhibkill', 'firstInhibitorAssist': 'firstinhibassist', 'dKills': 'doublekills', 'tKills': 'triplekills', 'qKills': 'quadrakills', 'pKills': 'pentakills', 'side':'side', 'gold':'goldearned', 'GoldSpent': 'goldspent', "pDamageDealt":"totalphysicaldamagedealt", "pDamageTaken":"physicaldamagetaken", 'mDamageDealt':'totalmagicdamagedealt', 'largestCriticalStrike':'largestcriticalstrike',  "largestKillingSpree":"largestkillingspree", "largestMultiKill":"largestmultikill", 'game-id':'gameId'}
scrape_headers = full_headers_dict.keys()
lpl_scrape_headers = lpl_full_headers_dict.keys()
table_headers = full_headers_dict.values()



dict_keys(['name', 'hero', 'kill', 'death', 'assist', 'firstBlood', 'totalDamageToChamp', 'pDamageToChamp', 'mDamageDealtToChamp', 'trueDamageDealtToChampions', 'damageDealtToObjectives', 'damageDealtToTurrets', 'totalHeal', 'totalDamageTaken', 'mDamageTaken', 'wardsKilled', 'sightWardsBoughtInGame', 'visionWardsBought', 'lasthit', 'neutralKilledTJungle', 'neutralKilledEJungle', 'killingSprees', 'longestTimeSpentLiving', 'damageSelfMitigated', 'visionScore', 'timeCCingOthers', 'towerKills', 'inhibitorKills', 'totalTimeCrowdControlDealt', 'firstBloodAssist', 'firstTowerKill', 'firstTowerAssist', 'firstInhibitorKill', 'firstInhibitorAssist', 'dKills', 'tKills', 'qKills', 'pKills', 'side', 'gold', 'GoldSpent', 'pDamageDealt', 'pDamageTaken', 'mDamageDealt', 'largestCriticalStrike', 'largestKillingSpree', 'largestMultiKill', 'game-id'])


In [3]:
y.keys()

dict_keys(['bMatchId', 'bMatchName', 'GameId', 'GameName', 'GameMode', 'GameModeName', 'GameTypeId', 'GameTypeName', 'GameProcId', 'GameProcName', 'TeamA', 'TeamNameA', 'ScoreA', 'TeamB', 'TeamNameB', 'ScoreB', 'MatchDate', 'MatchStatus', 'MatchWin', 'iQTMatchId', 'AppTopicId', 'isTft', 'tftInfos', 'bGameId', 'MatchTime', 'TeamIcon', 'sExt1', 'sExt1Src', 'sExt2', 'sExt2Src', 'sExt3Src', 'sExt3', 'sExt4Src', 'sExt4', 'iDefault', 'Video1', 'Video2', 'Video3', 'Chat1', 'Chat2', 'Chat3', 'News1', 'News2', 'News3', 'isFocus', 'sAfterNews', 'sMatchInfo', 'lastUpdateTime'])

In [9]:
bb['url'] = ['u', '']

[{'bMatchId': '6909',
  'sMatchId': '9183',
  'BattleId': '6542178',
  'AreaId': '134',
  'TeamId': '41',
  'MemberId': '592',
  'AccountId': '2627612370199904',
  'GameName': 'SNSofM',
  'Place': '5',
  'ChampionId': '76',
  'Game_K': '1',
  'Game_D': '1',
  'Game_A': '6',
  'Game_M': '10867',
  'Game_W': '1',
  'sUpdated': '2021-01-10 17:28:34',
  'iMVP': '0',
  'iDVPNum': '0'},
 {'bMatchId': '6909',
  'sMatchId': '9183',
  'BattleId': '6542178',
  'AreaId': '134',
  'TeamId': '41',
  'MemberId': '2268',
  'AccountId': '2625310050313568',
  'GameName': 'SNAngel',
  'Place': '2',
  'ChampionId': '112',
  'Game_K': '2',
  'Game_D': '0',
  'Game_A': '5',
  'Game_M': '12602',
  'Game_W': '1',
  'sUpdated': '2021-01-10 17:28:34',
  'iMVP': '0',
  'iDVPNum': '0'},
 {'bMatchId': '6909',
  'sMatchId': '9183',
  'BattleId': '6542178',
  'AreaId': '134',
  'TeamId': '41',
  'MemberId': '2284',
  'AccountId': '2625407631279456',
  'GameName': 'SNhuanfeng',
  'Place': '3',
  'ChampionId': '145',

In [ ]:
def get_data_lpl(json_content, test):
    list_list = []
    jd = json.loads(json_content["sMatchInfo"][0]['battleInfo']['BattleData']) #this returns a json
    keys = jd.keys()
    control = {'side':'left', 'count':0, 'records':0}
    gameId= json_content['bMatchId']
    #this subsection is to move between the 'right' and 'left' team lists because that's how this info is nested.
    while control['records'] < 10: #set this to 1 to get it to run for only one player
        output_list = []
        if control['count'] == 5:
            control['side'] = 'right'
            control['count'] = 0
        else: #I think the champion id -> champion modifier goes in here.
            side = control['side']
            count = control['count']
            data = jd[side]['players'][count] #dicts
        for i in a_keys:
            try:
                if i == 'hero': #this statement worked
                    output_list.append(str(champ_key_dict[jd[side]['players'][count][i]]))
                elif data[i]:
                    #print('yes')
                    output_list.append(data[i]) #list of all of the player info, which is currently fucking up
                elif i == 'firstBlood': #have to have this statement in here because apparently if the player didn't FB they just leave the line empty and python picks it up as False
                        if not data[i]:
                            data[i] = '0'
                            output_list.append(data[i])
                        else:
                            data[i] = '1'
                            output_list.append(data[i])
                elif i == 'game-id':
                    output_list.append(gameId)
                elif i == 'side':
                    output_list.append(control['side'])
                else:
                    print (i, 'not in lpl headers')
            except Exception as e: # probably dont need this full line because I'm doing something with the exception
                if i in b_keys:
                    print (i, "in non-lpl headers")
                    pass
                elif i == 'game-id' or 'side':
                    continue
                else:
                    print ("exception", i)
        list_list.append(output_list)
        control['count'] = control['count'] + 1
        control['records'] = control['records'] + 1
        print('done')
    if test == True:
        pprint.pprint(list_list)
    return(list_list)